# This is a jupyter notebook regarding all of the work on this final project for Information Retrieval

This is a GIF retrieval search system using annotated data, and the goal of this project is to get a solid retrieval system for gifs that properly action match.

## 1. First, we grab the simple initial annotated data as our "data source"

In [1]:
import json
file_path = "./sample_data.json"
with open(file_path, "r", encoding="utf-8") as file:
    simplified_data = json.load(file)

#show structure of the dataset (first few entries)
simplified_data[:3] if isinstance(simplified_data, list) else simplified_data

[{'query': 'slow-motion motorcycle crash',
  'narrative': 'this query should highlight a dramatic slow motion crash that clearly shows the motorcycle AND the crash, specifically in slow motion',
  'possible-results': [{'link': 'https://tenor.com/view/fail-bike-motorcycle-race-runaway-gif-3517863',
    'visual clarity': 1,
    'action matching': 0,
    'relevance': 0},
   {'link': 'https://tenor.com/view/ops-accident-motorcycle-racing-fall-gif-4939877',
    'visual clarity': 1,
    'action matching': 1,
    'relevance': 1}]},
 {'query': 'happy celebration dance',
  'narrative': 'this query should show someone or something (animate or in-animate object) performing some form of celebratory dance in a happy mood',
  'possible-results': [{'link': 'https://tenor.com/view/party-party-time-its-time-to-party-time-to-party-party-hard-gif-11255475805218535145',
    'visual clarity': 1,
    'action matching': 1,
    'relevance': 1},
   {'link': 'https://tenor.com/view/yay-seinfield-exciting-excite

## 2. Now that we have the essential data, let's try getting a baseline retrieval function with just query-keyword matching

In [2]:
#the most baseline retrieval that we can get, match query with the original query that we had
def baseline_retrieval(query, data):
    #split given query into words
    query_keywords = set(query.lower().split())
    results = []

    for entry in data:
        #right now, only data that we retrieve is the query text
        metadata_text = entry["query"].lower()
        metadata_keywords = set(metadata_text.split())

        #compute the raw overlap score with original query and given query
        match_score = len(query_keywords & metadata_keywords) / len(query_keywords)

        #store
        results.append({
            "query": entry["query"],
            "match_score": match_score,
            "results": entry["possible-results"]
        })

    #transform from nested structure to flat structure (creating a standard output format with the minimum information for now)
    formatted_results = []
    for result in results:
        for candidate in result["results"]:
            formatted_results.append({
                "parent_query": result["query"],
                "candidate_info": candidate,
                "score": result["match_score"]
            })

    #sort by the 'match score', highest first
    formatted_results.sort(key=lambda x: x["score"], reverse=True)

    return formatted_results

#first example of a query
test_query = "motorcycle crash slow motion"
baseline_results = baseline_retrieval(test_query, simplified_data)
baseline_results[:3]

[{'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://tenor.com/view/fail-bike-motorcycle-race-runaway-gif-3517863',
   'visual clarity': 1,
   'action matching': 0,
   'relevance': 0},
  'score': 0.5},
 {'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://tenor.com/view/ops-accident-motorcycle-racing-fall-gif-4939877',
   'visual clarity': 1,
   'action matching': 1,
   'relevance': 1},
  'score': 0.5},
 {'parent_query': 'happy celebration dance',
  'candidate_info': {'link': 'https://tenor.com/view/party-party-time-its-time-to-party-time-to-party-party-hard-gif-11255475805218535145',
   'visual clarity': 1,
   'action matching': 1,
   'relevance': 1},
  'score': 0.0}]

### With this initial retrieval, we can see that the ones that we wanted to retrieve (relatively) had a score of 0.5, but we purely gave the results without doing any other processing on it, so our "first result" had visual clarity, but no action-matching or relevance, compared to the second result which had all of the above.

## However, this super-simple data that was initially retrieved isn't going to cut it. Instead, we will use an automated system to extract all of the information from the Tenor API that we can get, leaving some room for manual evaluation (visual clarity, action matching, and relevance).

In [3]:
## UNCOMMENT THE FOLLOWING CODE TO COLLECT DATA FROM TENOR API
# # This code is commented out to avoid unnecessary API calls and rate limiting.

# import requests
# import json
# import time


# API_KEY = "" #the raw API key value from Google Cloud Console

# SEARCH_QUERIES = [
#     "slow-motion motorcycle crash",
#     "happy celebration dance",
#     "sad farewell waving",
#     "cat falling off table",
#     "sports goal celebration",
#     "funny reaction face",
#     "rainy day window view",
#     "mic drop moment",
#     "shocked expression wide eyes",
#     "explosive sneeze"
# ]
# LIMIT = 20

# def fetch_gifs(query, api_key, limit=20):
#     url = f"https://tenor.googleapis.com/v2/search"
#     params = {
#         "q": query,
#         "key": api_key,
#         "limit": limit,
#         "media_filter": "gif" #just filtering with "gifs", no mp4's tinygifs's etc
#     }
#     response = requests.get(url, params=params)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         print(f"Error fetching data for query: {query}, {response.text}")
#         return None

# collected_data = []

# for query in SEARCH_QUERIES:
#     print(f"Performing retrieval on \"{query}\"")
#     data = fetch_gifs(query, API_KEY, LIMIT)
#     if data:
#         #process each result as needed; here we simply collect a list of URLs and basic metadata
#         results = []
#         for item in data.get("results", []):
#             gif_information = item["media_formats"]["gif"] if "media_formats" in item else None
#             results.append({
#                 #link according to actual format
#                 "link": gif_information["url"],
#                 "duration": gif_information["duration"],
#                 "dimensions": gif_information["dims"],
#                 "size": gif_information["size"],
#                 "created-at": item["created"],
#                 "description": item["content_description"],
#                 "tags": item["tags"],
#                 "visual clarity": 0,  #placeholder for manual eval
#                 "action matching": 0,  #placeholder for manual eval
#                 "relevance": 0  #placeholder for manual eval
#             })
#         collected_data.append({
#             "query": query,
#             "narrative": f"Auto-generated narrative for {query}.",
#             "possible-results": results
#         })
#     #avoid rate limiting rq
#     time.sleep(1)

# #save all data into json
# with open("sample_data_extended.json", "w", encoding="utf-8") as outfile:
#     json.dump(collected_data, outfile, indent=4)

# print("Data collection complete!")

### Now, if we want to look at the annotated data (after we gave relevance judgements), we can do so in the following line

In [4]:
import json
file_path = "./sample_data_extended.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

#show structure of the dataset (first few entries)
data[:3] if isinstance(data, list) else data

[{'query': 'slow-motion motorcycle crash',
  'narrative': 'Auto-generated narrative for slow-motion motorcycle crash.',
  'possible-results': [{'link': 'https://media.tenor.com/R8glnb3Kcf0AAAAC/crash-landing-on-you-cloy.gif',
    'duration': 1.4,
    'dimensions': [498, 498],
    'size': 2291169,
    'created-at': 1582215148.108323,
    'description': 'a man is riding a motorcycle on the side of a road with chinese writing on it .',
    'tags': ['Crash Landing On You',
     'cloy',
     'Hyun Bin',
     'motorcycle',
     'biker',
     'slowly',
     'Slow Down',
     'beginner',
     'Captain Ri',
     'Ri Jeong Hyeok'],
    'visual clarity': 1,
    'action matching': 0,
    'relevance': 0},
   {'link': 'https://media.tenor.com/m9AggQGYXrgAAAAC/speeding-up-slow-motion.gif',
    'duration': 4.3,
    'dimensions': [498, 280],
    'size': 6650543,
    'created-at': 1564812306.733876,
    'description': 'dirt rider 42 is riding a dirt bike in the dirt',
    'tags': ['Speeding Up',
     'S

#### That should perform the exact same as the other baseline retrieval, where we simply retrieve all results that query-match it

In [5]:
test_query = "motorcycle crash slow motion"
baseline_results = baseline_retrieval(test_query, data)
baseline_results[:3]

[{'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://media.tenor.com/R8glnb3Kcf0AAAAC/crash-landing-on-you-cloy.gif',
   'duration': 1.4,
   'dimensions': [498, 498],
   'size': 2291169,
   'created-at': 1582215148.108323,
   'description': 'a man is riding a motorcycle on the side of a road with chinese writing on it .',
   'tags': ['Crash Landing On You',
    'cloy',
    'Hyun Bin',
    'motorcycle',
    'biker',
    'slowly',
    'Slow Down',
    'beginner',
    'Captain Ri',
    'Ri Jeong Hyeok'],
   'visual clarity': 1,
   'action matching': 0,
   'relevance': 0},
  'score': 0.5},
 {'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://media.tenor.com/m9AggQGYXrgAAAAC/speeding-up-slow-motion.gif',
   'duration': 4.3,
   'dimensions': [498, 280],
   'size': 6650543,
   'created-at': 1564812306.733876,
   'description': 'dirt rider 42 is riding a dirt bike in the dirt',
   'tags': ['Speeding Up',
    'Slow Motion',
 

## Now, we have a very basic model, but it only is based on the initial query search. Let's improve it by appending all of the wording-based metadata values together, and computing that overlap as the "score". This will differentiate by the result, and be based on more than the overarching query!

In [6]:
def concatenator_retrieval(query, data):
    def compute_score_candidate(query_keywords, candidate_query, candidate):
        #for this function, we have to concatenate all of the candidate information together,
        # and calculate the overlap between them
        
        #AS A NOTE, WE ONLY RETRIEVE THE DESCRIPTION AND TAGS HERE, JUST FOR SIMPLICITY OF CONCATENATION/SCORING
        candidate_text = f"{candidate_query} {candidate.get('description', '')} {' '.join(candidate.get('tags', []))}"
        candidate_keywords = set(candidate_text.lower().split())
        
        #overlap them
        overlap_score = len(query_keywords & candidate_keywords) / len(query_keywords)
        return overlap_score

    #split given query into words
    query_keywords = set(query.lower().split())
    flat_results = []

    for query_block in data:
        query_text = query_block["query"]
        candidates = query_block.get("possible-results", [])

        #for each candidate, compute the score based on the concatenated text
        for candidate in candidates:
            score = compute_score_candidate(query_keywords, query_text, candidate)
            flat_results.append({
                "parent_query": query_text,
                "candidate_info": candidate,
                "score": score
            })

    sorted_results = sorted(flat_results, key=lambda x: x["score"], reverse=True)
    return sorted_results

#first example of a query
test_query = "motorcycle crash slow motion"
baseline_results = concatenator_retrieval(test_query, data)
baseline_results[:3]

[{'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://media.tenor.com/m9AggQGYXrgAAAAC/speeding-up-slow-motion.gif',
   'duration': 4.3,
   'dimensions': [498, 280],
   'size': 6650543,
   'created-at': 1564812306.733876,
   'description': 'dirt rider 42 is riding a dirt bike in the dirt',
   'tags': ['Speeding Up',
    'Slow Motion',
    'motocross',
    'Motocross Bike Reviews',
    'Ktm450Sxf',
    'Dirt Rider'],
   'visual clarity': 1,
   'action matching': 0,
   'relevance': 0},
  'score': 1.0},
 {'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://media.tenor.com/Sw-m4TsNYB8AAAAC/slowmo-sliding.gif',
   'duration': 4.4,
   'dimensions': [498, 280],
   'size': 6213479,
   'created-at': 1542564060.293962,
   'description': 'a close up of a person riding a motorcycle with smoke coming out of the exhaust pipe .',
   'tags': ['slowmo',
    'sliding',
    'cr85',
    'racing',
    'mini',
    'motorcycles',
    'cr80',

### As you can see, this gives us a good display that we retrieve only the data that we need, yet we struggle to differentiate between the different fields. We don't want a singular "tag" to be held to the same standard as a word from the query, so let's add some weights, and consider the fields separately from one another.

In [7]:
def weighted_concatenator_retrieval(query, data, weights=None):
    if weights is None:
        #default weights for candidate_query, description, and tags
        weights = {"candidate_query": 0.5, "description": 0.3, "tags": 0.2}

    def compute_weighted_score(query_keywords, candidate_query, candidate):
        #extract the fields
        candidate_query_text = candidate_query.lower()
        description_text = candidate.get("description", "").lower()
        tags_text = " ".join(candidate.get("tags", [])).lower()

        #split each into keywords
        candidate_query_keywords = set(candidate_query_text.split())
        description_keywords = set(description_text.split())
        tags_keywords = set(tags_text.split())

        #compute the individual overlap scores
        query_overlap = len(query_keywords & candidate_query_keywords) / len(query_keywords)
        description_overlap = len(query_keywords & description_keywords) / len(query_keywords)
        tags_overlap = len(query_keywords & tags_keywords) / len(query_keywords)

        #multiply scores with weights
        weighted_score = (
            weights["candidate_query"] * query_overlap +
            weights["description"] * description_overlap +
            weights["tags"] * tags_overlap
        )
        return weighted_score

    #split query
    query_keywords = set(query.lower().split())
    flat_results = []

    for query_block in data:
        query_text = query_block["query"]
        candidates = query_block.get("possible-results", [])

        #score all candidates
        for candidate in candidates:
            score = compute_weighted_score(query_keywords, query_text, candidate)
            flat_results.append({
                "parent_query": query_text,
                "candidate_info": candidate,
                "score": score
            })

    sorted_results = sorted(flat_results, key=lambda x: x["score"], reverse=True)
    return sorted_results

test_query = "motorcycle crash slow motion"
weighted_results = weighted_concatenator_retrieval(test_query, data)
weighted_results[:3]

[{'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://media.tenor.com/Sw-m4TsNYB8AAAAC/slowmo-sliding.gif',
   'duration': 4.4,
   'dimensions': [498, 280],
   'size': 6213479,
   'created-at': 1542564060.293962,
   'description': 'a close up of a person riding a motorcycle with smoke coming out of the exhaust pipe .',
   'tags': ['slowmo',
    'sliding',
    'cr85',
    'racing',
    'mini',
    'motorcycles',
    'cr80',
    'honda',
    'trackday',
    'supermoto',
    'minis',
    'crash',
    'motorcycle',
    'slide',
    'track',
    'slow',
    'motion'],
   'visual clarity': 1,
   'action matching': 0,
   'relevance': 0},
  'score': 0.525},
 {'parent_query': 'slow-motion motorcycle crash',
  'candidate_info': {'link': 'https://media.tenor.com/R8glnb3Kcf0AAAAC/crash-landing-on-you-cloy.gif',
   'duration': 1.4,
   'dimensions': [498, 498],
   'size': 2291169,
   'created-at': 1582215148.108323,
   'description': 'a man is riding a motorcycle on

In [8]:
#really quickly, we should define what we want to measure for the models to compare them
#in my case, I would like to maximize the recall@k values and MAP@k (mean average precision) score

# recall@k = number of relevant items retrieved in top k results / total number of relevant items
# MAP@k = mean of the average precision scores for each query, averaged over all queries

#now, we can define a function to compute the recall@k and MAP@k values for our retrieval models
def compute_recall_and_map(retrieval_results, ground_truth, k=5):
    """
    params:
    retrieval_results: List of retrieval results (sorted by score).
    ground_truth: List of relevant items for the query.
    k: Number of top results to consider for recall and MAP.
    
    returns:
    recall_at_k: Recall@k value.
    map_at_k: MAP@k value.
    """
    #get the top k results
    top_k_results = [result["candidate_info"]["link"] for result in retrieval_results[:k]]
    
    #calculate recall@k
    relevant_retrieved = len(set(top_k_results) & set(ground_truth))
    total_relevant = len(ground_truth)
    recall_at_k = relevant_retrieved / total_relevant if total_relevant > 0 else 0
    #^literally just the relevant retrieved divided by the total relevant items
    
    #calculate MAP@k
    average_precision = 0
    for i, result in enumerate(top_k_results):
        if result in ground_truth:
            average_precision += (i + 1) / (i + 1)  #precision at i+1
            
    map_at_k = average_precision / min(k, total_relevant) if total_relevant > 0 else 0
    #^mean of the average precision scores for each query, averaged over all queries
    #this is a bit of a simplification, but it should work for our general purposes
    
    return recall_at_k, map_at_k

#now that we have that, let's define a function to compute the recall and MAP for any retrieval model
#given a retrieval model, the raw dataset, a test query, and the k value, calculate the MAP and recall values

def evaluate_all_queries(retrieval_model, data, test_queries, query_mapping, k=5, verbose=False):
    """
    params:
    retrieval_model: Function that takes a query and data, and returns sorted retrieval results.
    data: Dataset containing the queries and their relevant items.
    test_query: The specific query to evaluate.
    k: Number of top results to consider for recall and MAP.
    
    returns:
    evaluation: Dictionary containing recall@k and MAP@k for the test query.
    """
    evaluations = []
    #find the ground truth relevant items for the test query
    for test_query in test_queries:
        #map the test query to its corresponding "grouth truth" query in the dataset
        training_query = query_mapping.get(test_query)
        if not training_query:
            raise ValueError(f"Test query '{test_query}' not found in query mapping.")

        ground_truth = []
        for entry in data:
            if entry["query"] == training_query:
                #not all values in the possible-results are relevant, so we only add the relevant ones
                ground_truth = [result["link"] for result in entry.get("possible-results", []) if result["relevance"] == 1]
                break

        if not ground_truth:
            #if we don't find any relevant items, we can't compute recall or MAP [specifically for the slow-motion motorcycle crash query]
            if verbose:
                print(f"No relevant items found for training query '{training_query}'.")
            continue

        #get the retrieval results for the test query (NOT THE TRAINING QUERY)
        retrieval_results = retrieval_model(test_query, data)

        #compute recall and MAP
        recall_at_k, map_at_k = compute_recall_and_map(retrieval_results, ground_truth, k)

        #store the evaluation results
        evaluation = {
            "query": test_query,
            "recall_at_k": recall_at_k,
            "map_at_k": map_at_k
        }
        evaluations.append(evaluation)
    
    #calculate the average recall and MAP across all queries
    avg_recall = sum(evaluation["recall_at_k"] for evaluation in evaluations) / len(evaluations)
    avg_map = sum(evaluation["map_at_k"] for evaluation in evaluations) / len(evaluations)

    return {
        "average_recall_at_k": avg_recall,
        "average_map_at_k": avg_map,
        "individual_evaluations": evaluations
    }

In [9]:
#let's measure the recall and MAP for Tenor's retrieval system (by just using the "ground truth" values) and our own retrieval system
#we will use the same test queries as the ones we used to collect data from Tenor API
#and we will use the same k value as well (5)
#we will also use the same query mapping as before, to map the test queries to their corresponding training queries in the dataset

TENOR_ONLY_SEARCH_QUERIES = [
    "slow-motion motorcycle crash",
    "happy celebration dance",
    "sad farewell waving",
    "cat falling off table",
    "sports goal celebration",
    "funny reaction face",
    "rainy day window view",
    "mic drop moment",
    "shocked expression wide eyes",
    "explosive sneeze"
]

#mapping of test query to test query in the dataset
TENOR_ONLY_query_mapping = {
    "slow-motion motorcycle crash": "slow-motion motorcycle crash",
    "happy celebration dance": "happy celebration dance",
    "sad farewell waving": "sad farewell waving",
    "cat falling off table": "cat falling off table",
    "sports goal celebration": "sports goal celebration",
    "funny reaction face": "funny reaction face",
    "rainy day window view": "rainy day window view",
    "mic drop moment": "mic drop moment",
    "shocked expression wide eyes": "shocked expression wide eyes",
    "explosive sneeze": "explosive sneeze"
}

def tenor_retrieval(query, data):
    """
    This function simulates the retrieval system of Tenor by using the original query as the retrieval query.
    It returns the results from the dataset based on the original query.
    """
    #find the entry in the dataset that matches the query
    for entry in data:
        if entry["query"] == query:
            #format the results to include the required information (same as baseline_retrieval)
            formatted_results = [
                {
                    "parent_query": query,
                    "candidate_info": result,  # Include the full result as "candidate_info"
                    "score": 1.0  # Assign a default score of 1.0 for Tenor's results
                }
                for result in entry.get("possible-results", [])
            ]
            return formatted_results
    return [] #if theres no matching query, return an empty list

#evaluate tenor's retrieval system
tenor_evaluation = evaluate_all_queries(tenor_retrieval, data, TENOR_ONLY_SEARCH_QUERIES, TENOR_ONLY_query_mapping, k=5)
print("Tenor Retrieval Baseline Evaluation:")
print(f"Average Recall@5: {tenor_evaluation['average_recall_at_k']:.4f}")
print(f"Average MAP@5: {tenor_evaluation['average_map_at_k']:.4f}")

#print individual query evaluations (so we can see how well tenor is doing on each query)
print("\nIndividual Query Evaluations:")
for evaluation in tenor_evaluation["individual_evaluations"]:
    print(f"Query: {evaluation['query']} | Recall@5: {evaluation['recall_at_k']:.4f} | MAP@5: {evaluation['map_at_k']:.4f}")

Tenor Retrieval Baseline Evaluation:
Average Recall@5: 0.2500
Average MAP@5: 0.8000

Individual Query Evaluations:
Query: happy celebration dance | Recall@5: 0.2222 | MAP@5: 0.8000
Query: sad farewell waving | Recall@5: 0.1875 | MAP@5: 0.6000
Query: cat falling off table | Recall@5: 0.2353 | MAP@5: 0.8000
Query: sports goal celebration | Recall@5: 0.2941 | MAP@5: 1.0000
Query: funny reaction face | Recall@5: 0.2000 | MAP@5: 0.6000
Query: rainy day window view | Recall@5: 0.3125 | MAP@5: 1.0000
Query: mic drop moment | Recall@5: 0.2308 | MAP@5: 0.6000
Query: shocked expression wide eyes | Recall@5: 0.2105 | MAP@5: 0.8000
Query: explosive sneeze | Recall@5: 0.3571 | MAP@5: 1.0000


## Now that we've quickly defined the actual metrics that we want to consider, and that we have Tenor's baseline accuracy, let's measure the current values for the models that we built and see if they improve!

In [10]:
#all of these queries are slightly different from the original queries, but they are still relevant to the original query
#we can use these queries to evaluate the retrieval models
TEST_SEARCH_QUERIES = [
    "motorcycle accident in slow motion",
    "celebration dance with joy",
    "waving goodbye sadly",
    "cat falling from a table",
    "goal celebration in sports",
    "funny facial reaction",
    "looking out a rainy window",
    "dropping the mic moment",
    "wide-eyed shocked expression",
    "big sneeze explosion"
]

#heres a direct query mapping from the original queries to the new queries (we only use this to get the "grouth truth" values, not for the retrieval models themselves)
query_mapping = {
    "motorcycle accident in slow motion": "slow-motion motorcycle crash",
    "celebration dance with joy": "happy celebration dance",
    "waving goodbye sadly": "sad farewell waving",
    "cat falling from a table": "cat falling off table",
    "goal celebration in sports": "sports goal celebration",
    "funny facial reaction": "funny reaction face",
    "looking out a rainy window": "rainy day window view",
    "dropping the mic moment": "mic drop moment",
    "wide-eyed shocked expression": "shocked expression wide eyes",
    "big sneeze explosion": "explosive sneeze"
}

baseline_evaluation = evaluate_all_queries(baseline_retrieval, data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("Baseline Retrieval Evaluation:")
print(f"Average Recall@5: {baseline_evaluation['average_recall_at_k']:.4f}")
print(f"Average MAP@5: {baseline_evaluation['average_map_at_k']:.4f}")
#we still have access to individual evaluations per query, but we wont print those out

concatenator_evaluation = evaluate_all_queries(concatenator_retrieval, data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("Concatenator Retrieval Evaluation:")
print(f"Average Recall@5: {concatenator_evaluation['average_recall_at_k']:.4f}")
print(f"Average MAP@5: {concatenator_evaluation['average_map_at_k']:.4f}")

weighted_evaluation = evaluate_all_queries(weighted_concatenator_retrieval, data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("Weighted Concatenator Retrieval Evaluation:")
print(f"Average Recall@5: {weighted_evaluation['average_recall_at_k']:.4f}")
print(f"Average MAP@5: {weighted_evaluation['average_map_at_k']:.4f}")

Baseline Retrieval Evaluation:
Average Recall@5: 0.2500
Average MAP@5: 0.8000
Concatenator Retrieval Evaluation:
Average Recall@5: 0.2441
Average MAP@5: 0.7778
Weighted Concatenator Retrieval Evaluation:
Average Recall@5: 0.2468
Average MAP@5: 0.8000


## Now that we have the basic evaluation metrics, let's do hyperparameter search with the weighted evaluation!

In [11]:
#we could technically visualize the results, but I just want to sort them by the recall and MAP values
#and see which weights performed the best
import itertools

def hyperparameter_search(data, test_queries, query_mapping, k=5):
    #generate all possible combinations of weights for the weighted concatenator retrieval
    #NOTE: they have to sum to 1, and be between 0 and 1
    #this is a brute-force approach, but it should work for our purposes
    weight_combinations = []
    for weights in itertools.product(range(0, 11), repeat=3):
        total = sum(weights)
        if total == 10:  #weights have to sum to 1 (in this case, we just multiply by 0.1)
            weight_combinations.append(tuple(weight / 10 for weight in weights))

    #store the results for each weight combination
    results = []
    for weights in weight_combinations:

        formatted_weights = {
            "candidate_query": weights[0],
            "description": weights[1],
            "tags": weights[2]
        }

        #define a custom weighted model with the given weights
        def weighted_model(query, data):
                return weighted_concatenator_retrieval(query, data, formatted_weights)
        
        #evaluate the retrieval model with the given weights
        evaluation = evaluate_all_queries(weighted_model, data, test_queries, query_mapping, k=k)
        results.append({
            "weights": formatted_weights,
            "average_recall_at_k": evaluation["average_recall_at_k"],
            "average_map_at_k": evaluation["average_map_at_k"]
        })

    #sort the results by recall and MAP
    sorted_results = sorted(results, key=lambda x: (x["average_recall_at_k"], x["average_map_at_k"]), reverse=True)
    return sorted_results

#run the hyperparameter search
hyperparameter_results = hyperparameter_search(data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("Hyperparameter Search Results (Top 10):")
for i, result in enumerate(hyperparameter_results[:10]):
    print(f"Rank {i+1}: Weights: {result['weights']}, Average Recall@5: {result['average_recall_at_k']:.4f}, Average MAP@5: {result['average_map_at_k']:.4f}")

Hyperparameter Search Results (Top 10):
Rank 1: Weights: {'candidate_query': 0.6, 'description': 0.4, 'tags': 0.0}, Average Recall@5: 0.2637, Average MAP@5: 0.8444
Rank 2: Weights: {'candidate_query': 0.7, 'description': 0.3, 'tags': 0.0}, Average Recall@5: 0.2637, Average MAP@5: 0.8444
Rank 3: Weights: {'candidate_query': 0.8, 'description': 0.2, 'tags': 0.0}, Average Recall@5: 0.2637, Average MAP@5: 0.8444
Rank 4: Weights: {'candidate_query': 0.9, 'description': 0.1, 'tags': 0.0}, Average Recall@5: 0.2637, Average MAP@5: 0.8444
Rank 5: Weights: {'candidate_query': 0.4, 'description': 0.5, 'tags': 0.1}, Average Recall@5: 0.2617, Average MAP@5: 0.8444
Rank 6: Weights: {'candidate_query': 0.5, 'description': 0.4, 'tags': 0.1}, Average Recall@5: 0.2617, Average MAP@5: 0.8444
Rank 7: Weights: {'candidate_query': 0.6, 'description': 0.3, 'tags': 0.1}, Average Recall@5: 0.2617, Average MAP@5: 0.8444
Rank 8: Weights: {'candidate_query': 0.4, 'description': 0.4, 'tags': 0.2}, Average Recall@5

## Sweet! We have a much better recall and MAP for the weighted values!! Now, let's actually complete the remaining TODOs:
- rather than pure overlap, we can also use a more advanced scoring function (bm25, tf-idf, etc.),
 or even a neural network model to compute the score (e.g., BERT, etc.) [much more advanced, and reach]
- also, we can use the same scoring function for the baseline/concatenator retrieval as well
(also make sure to separate the train/test data, and not use the same data for training and testing)
- draw conclusions

In [12]:
#okay, now that we've gotten the best results for the raw overlap scoring methods for the retrieval models,
#let's see if we can improve the results with different scoring methods and models!

#NEW SCORING METHOD
def jaccard_similarity(query_keywords, candidate_keywords):
    #compute the Jaccard similarity between two sets of keywords
    intersection = len(query_keywords & candidate_keywords)
    union = len(query_keywords | candidate_keywords)
    return intersection / union if union > 0 else 0

#NEW SCORING METHOD
def cosine_similarity(query_keywords, candidate_keywords):
    #compute the cosine similarity between two sets of keywords
    intersection = len(query_keywords & candidate_keywords)
    return intersection / (len(query_keywords) * len(candidate_keywords)) ** 0.5 if len(query_keywords) > 0 and len(candidate_keywords) > 0 else 0

#NEW SCORING METHOD
def bm25_similarity(query_keywords, candidate_keywords):
    #compute the BM25 similarity between two sets of keywords
    k1 = 1.5
    b = 0.75
    avg_doc_length = len(candidate_keywords)  #average document length (in this case, just the length of the candidate keywords)
    doc_length = len(candidate_keywords)  #length of the current document (candidate keywords)

    #convert the list to allow for counting
    candidate_keywords_list = list(candidate_keywords)

    score = 0
    for keyword in query_keywords:
        if keyword in candidate_keywords_list:
            tf = candidate_keywords_list.count(keyword)  #term frequency in the candidate
            idf = len(candidate_keywords_list) / (1 + candidate_keywords_list.count(keyword))  #inverse document frequency
            score += idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_length / avg_doc_length))))

    return score

#helper function to redirect the similarity computation to the right function
#this is just a wrapper function to call the right similarity function based on the method specified
def compute_similarity(query_keywords, candidate_keywords, method="jaccard"):
    if method == "jaccard":
        return jaccard_similarity(query_keywords, candidate_keywords)
    elif method == "cosine":
        return cosine_similarity(query_keywords, candidate_keywords)
    elif method == "bm25":
        return bm25_similarity(query_keywords, candidate_keywords)
    else:
        raise ValueError(f"Unknown similarity method: {method}")

#helper function to compute the weighted score with similarity
def compute_weighted_score_with_similarity(query_keywords, candidate_query, candidate, weights, method="jaccard"):
    #extract the fields
    candidate_query_text = candidate_query.lower()
    description_text = candidate.get("description", "").lower()
    tags_text = " ".join(candidate.get("tags", [])).lower()

    #split each into keywords
    candidate_query_keywords = set(candidate_query_text.split())
    description_keywords = set(description_text.split())
    tags_keywords = set(tags_text.split())

    #compute the individual overlap scores
    query_overlap = compute_similarity(query_keywords, candidate_query_keywords, method)
    description_overlap = compute_similarity(query_keywords, description_keywords, method)
    tags_overlap = compute_similarity(query_keywords, tags_keywords, method)

    #multiply scores with weights
    weighted_score = (
        weights["candidate_query"] * query_overlap +
        weights["description"] * description_overlap +
        weights["tags"] * tags_overlap
    )
    return weighted_score

In [13]:
#let's define new retrieval models that use the new scoring methods (keeping the same structure as before)

#NEW MODEL(USING JACCARD SIMILARITY)
def jaccard_weighted_retrieval(query, data, weights=None):
    if weights is None:
        #default weights for candidate_query, description, and tags
        weights = {"candidate_query": 0.5, "description": 0.3, "tags": 0.2}

    def compute_weighted_score(query_keywords, candidate_query, candidate):
        return compute_weighted_score_with_similarity(query_keywords, candidate_query, candidate, weights, method="jaccard")

    #split query
    query_keywords = set(query.lower().split())
    flat_results = []

    for query_block in data:
        query_text = query_block["query"]
        candidates = query_block.get("possible-results", [])

        #score all candidates
        for candidate in candidates:
            score = compute_weighted_score(query_keywords, query_text, candidate)
            flat_results.append({
                "parent_query": query_text,
                "candidate_info": candidate,
                "score": score
            })

    sorted_results = sorted(flat_results, key=lambda x: x["score"], reverse=True)
    return sorted_results

#NEW MODEL(USING COSINE SIMILARITY)
def cosine_weighted_retrieval(query, data, weights=None):
    if weights is None:
        #default weights for candidate_query, description, and tags
        weights = {"candidate_query": 0.5, "description": 0.3, "tags": 0.2}

    def compute_weighted_score(query_keywords, candidate_query, candidate):
        return compute_weighted_score_with_similarity(query_keywords, candidate_query, candidate, weights, method="cosine")

    #split query
    query_keywords = set(query.lower().split())
    flat_results = []

    for query_block in data:
        query_text = query_block["query"]
        candidates = query_block.get("possible-results", [])

        #score all candidates
        for candidate in candidates:
            score = compute_weighted_score(query_keywords, query_text, candidate)
            flat_results.append({
                "parent_query": query_text,
                "candidate_info": candidate,
                "score": score
            })

    sorted_results = sorted(flat_results, key=lambda x: x["score"], reverse=True)
    return sorted_results

#NEW MODEL(USING BM25 SIMILARITY)
def bm25_weighted_retrieval(query, data, weights=None):
    if weights is None:
        #default weights for candidate_query, description, and tags
        weights = {"candidate_query": 0.5, "description": 0.3, "tags": 0.2}

    def compute_weighted_score(query_keywords, candidate_query, candidate):
        return compute_weighted_score_with_similarity(query_keywords, candidate_query, candidate, weights, method="bm25")

    #split query
    query_keywords = set(query.lower().split())
    flat_results = []

    for query_block in data:
        query_text = query_block["query"]
        candidates = query_block.get("possible-results", [])

        #score all candidates
        for candidate in candidates:
            score = compute_weighted_score(query_keywords, query_text, candidate)
            flat_results.append({
                "parent_query": query_text,
                "candidate_info": candidate,
                "score": score
            })

    sorted_results = sorted(flat_results, key=lambda x: x["score"], reverse=True)
    return sorted_results

In [14]:
#lets evaluate the default weights for the new retrieval models
jaccard_evaluation = evaluate_all_queries(jaccard_weighted_retrieval, data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("Jaccard Weighted Retrieval Evaluation:")
print(f"Average Recall@5: {jaccard_evaluation['average_recall_at_k']:.4f}")
print(f"Average MAP@5: {jaccard_evaluation['average_map_at_k']:.4f}")

cosine_evaluation = evaluate_all_queries(cosine_weighted_retrieval, data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("Cosine Weighted Retrieval Evaluation:")
print(f"Average Recall@5: {cosine_evaluation['average_recall_at_k']:.4f}")
print(f"Average MAP@5: {cosine_evaluation['average_map_at_k']:.4f}")

bm25_evaluation = evaluate_all_queries(bm25_weighted_retrieval, data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("BM25 Weighted Retrieval Evaluation:")
print(f"Average Recall@5: {bm25_evaluation['average_recall_at_k']:.4f}")
print(f"Average MAP@5: {bm25_evaluation['average_map_at_k']:.4f}")

Jaccard Weighted Retrieval Evaluation:
Average Recall@5: 0.2781
Average MAP@5: 0.8889
Cosine Weighted Retrieval Evaluation:
Average Recall@5: 0.2781
Average MAP@5: 0.8889
BM25 Weighted Retrieval Evaluation:
Average Recall@5: 0.2026
Average MAP@5: 0.6667


In [15]:
#let's perform hyperparameter search, similar to the weighted one from the concatenator retrieval, on all the new models
#and rank them by the recall and MAP values

#each result should display the rank, weights, method used, recall, and MAP values

def hyperparameter_search_similarity(data, test_queries, query_mapping, k=5):
    #generate all possible combinations of weights for the weighted concatenator retrieval
    weight_combinations = []
    for weights in itertools.product(range(0, 11), repeat=3):
        total = sum(weights)
        if total == 10:  #weights have to sum to 1 (in this case, we just multiply by 0.1)
            weight_combinations.append(tuple(weight / 10 for weight in weights))

    #store the results for each weight combination
    results = []
    for weights in weight_combinations:

        formatted_weights = {
            "candidate_query": weights[0],
            "description": weights[1],
            "tags": weights[2]
        }

        #define a custom weighted model with the given weights
        def jaccard_model(query, data):
                return jaccard_weighted_retrieval(query, data, formatted_weights)

        def cosine_model(query, data):
                return cosine_weighted_retrieval(query, data, formatted_weights)

        def bm25_model(query, data):
                return bm25_weighted_retrieval(query, data, formatted_weights)

        #evaluate the retrieval model with the given weights
        evaluation_jaccard = evaluate_all_queries(jaccard_model, data, test_queries, query_mapping, k=k)
        evaluation_cosine = evaluate_all_queries(cosine_model, data, test_queries, query_mapping, k=k)
        evaluation_bm25 = evaluate_all_queries(bm25_model, data, test_queries, query_mapping, k=k)

        #add the jaccard, cosine, and bm25 evaluations separately from one another
        results.append({
            "weights": formatted_weights,
            "method": "jaccard",
            "average_recall_at_k": evaluation_jaccard["average_recall_at_k"],
            "average_map_at_k": evaluation_jaccard["average_map_at_k"]
        })
        results.append({
            "weights": formatted_weights,
            "method": "cosine",
            "average_recall_at_k": evaluation_cosine["average_recall_at_k"],
            "average_map_at_k": evaluation_cosine["average_map_at_k"]
        })
        results.append({
            "weights": formatted_weights,
            "method": "bm25",
            "average_recall_at_k": evaluation_bm25["average_recall_at_k"],
            "average_map_at_k": evaluation_bm25["average_map_at_k"]
        })

    #sort the results by recall and MAP
    sorted_results = sorted(results, key=lambda x: (x["average_recall_at_k"], x["average_map_at_k"]), reverse=True)
    return sorted_results

#run the hyperparameter search
hyperparameter_results_similarity = hyperparameter_search_similarity(data, TEST_SEARCH_QUERIES, query_mapping, k=5)
print("Hyperparameter Search Results (Top 10):")
for i, result in enumerate(hyperparameter_results_similarity[:10]):
    print(f"Rank {i+1}: Weights: {result['weights']}, Method: {result['method']}, Average Recall@5: {result['average_recall_at_k']:.4f}, Average MAP@5: {result['average_map_at_k']:.4f}")

Hyperparameter Search Results (Top 10):
Rank 1: Weights: {'candidate_query': 0.2, 'description': 0.7, 'tags': 0.1}, Method: jaccard, Average Recall@5: 0.2855, Average MAP@5: 0.9111
Rank 2: Weights: {'candidate_query': 0.2, 'description': 0.7, 'tags': 0.1}, Method: cosine, Average Recall@5: 0.2855, Average MAP@5: 0.9111
Rank 3: Weights: {'candidate_query': 0.3, 'description': 0.6, 'tags': 0.1}, Method: jaccard, Average Recall@5: 0.2855, Average MAP@5: 0.9111
Rank 4: Weights: {'candidate_query': 0.3, 'description': 0.6, 'tags': 0.1}, Method: cosine, Average Recall@5: 0.2855, Average MAP@5: 0.9111
Rank 5: Weights: {'candidate_query': 0.4, 'description': 0.5, 'tags': 0.1}, Method: jaccard, Average Recall@5: 0.2855, Average MAP@5: 0.9111
Rank 6: Weights: {'candidate_query': 0.4, 'description': 0.5, 'tags': 0.1}, Method: cosine, Average Recall@5: 0.2855, Average MAP@5: 0.9111
Rank 7: Weights: {'candidate_query': 0.5, 'description': 0.4, 'tags': 0.1}, Method: jaccard, Average Recall@5: 0.2855

Again, we got really good results for the MAP! It looks like the BM25 does perform super poorly in comparison to the others, so we won't use BM25 score anymore. 

Also, it looks to be as though the tags are less important than we thought, and that when we include more weight to the tags, it ends up tanking the performance of the MAP results. So honestly, its just a skill issue on Tenor's part for ONLY considering the tags.

In [16]:
#my final test, I'll be using the jacard similarity with the optimized weights on a query specifying some key features
#of the image, and I want to see if my system could actually retrieve it.

#for this example, I'll be looking for the gif of: https://media.tenor.com/8md20a_wndwAAAAC/out-mic-drop.gif
#the query that I'll use will be "i'm out red guy" since that might be the only feature that i remember

print("Final Test Query:")
test_query = "i'm out red guy"
#use the jaccard weighted retrieval model with the optimized weights
optimized_weights = {
    "candidate_query": 0.2,
    "description": 0.7,
    "tags": 0.1
}
#run the retrieval model with the optimized weights
final_results = jaccard_weighted_retrieval(test_query, data, optimized_weights)

print("Top result:")
print(f"Link: {final_results[0]['candidate_info']['link']}")
print(f"Description: {final_results[0]['candidate_info']['description']}")

Final Test Query:
Top result:
Link: https://media.tenor.com/8md20a_wndwAAAAC/out-mic-drop.gif
Description: a man in a red shirt is holding a microphone in his mouth and says `` i 'm out ! ''


and it matches!!

# Conclusion on this project:

The best retrieval system, which ended up being tied 8 different ways, turned out to be Jaccard/Cosine similarity with the following weights put on the individual pieces:
- The candidate query the gif was based off of: 0.2
- The auto-generated or manually entered description: 0.7
- The tags themselves: 0.1

The "key features" happened to be the features that Tenor doesn't actually use, but I could see why tenor decided not to use it. After all, it does take a good bit of processing to produce results that didn't have the generated description, and it's easier to do a similarity match on the tags themselves. Especially with the portability of it, it makes sense to reduce these calculations, but if you wanted "actually good results", I would have expected them to take that tradeoff of expensive processing.

In [17]:
#for your personal use, I made a quick function to, using the optimized weights, retrieve the top 5 results for any given query
def retrieve_top_k_results(query, k):
    #optimal weights
    weights = {"candidate_query": 0.2, "description": 0.7, "tags": 0.1}

    #use the jaccard weighted retrieval model with the optimized weights
    results = jaccard_weighted_retrieval(query, data, weights)

    #return the top 5 results
    return results[:k]

#show the top 5 gifs for each query, like actually displaying them using IPython.display
from IPython.display import Image, display, HTML
def display_top_k_results(query, k):
    results = retrieve_top_k_results(query, k)
    for result in results:
        display(HTML(f"<h2>Top {k} results for query: {query}</h2>"))
        display(HTML(f"<a href='{result['candidate_info']['link']}'>{result['candidate_info']['description']}</a>"))
        display(Image(url=result['candidate_info']['link'], width=200, height=200))

#example usage (we only show the first 5 results, but you can change the number of results to show)
display_top_k_results("i'm out red guy", k=5)
